# Imports

In [3]:
import os
import json

import requests
import pandas as pd

from typing import Optional
from dotenv import load_dotenv

In [2]:
# Access environment variables
ACCOUNT_ID = os.getenv("ACCOUNT_ID")
BOOMI_USER = os.getenv("BOOMI_USER")
ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")

print(ACCOUNT_ID)
print(BOOMI_USER)
print(ACCESS_TOKEN)

personal-K8JJK1
andy.strubhar@argano.com
14592d8f-25a0-4d20-97e5-a0a7e17eb49f


# Set Default Headers and Auth

In [3]:
account_id = "personal-K8JJK1"
base_url = f"https://api.boomi.com/api/rest/v1/{account_id}"

default_headers = {
    "Content-Type": "application/json",
    "Accept":"application/json",
    "user":"BOOMI_TOKEN.andy.strubhar@argano.com:14592d8f-25a0-4d20-97e5-a0a7e17eb49f",
    "Connection": "close"
    }

# Function Declerations

## Get Env Vars - getEnvironmentVariables

In [5]:
def getEnvironmentVariables() -> dict:
    '''
    Gets environment variables from a .env file in the same directory as the script containing this function

    Returns
    -------
    dict
        A dictionary of environment vars, with the following form
            {
            "ACCOUNT_ID": ACCOUNT_ID,
            "BOOMI_USER": BOOMI_USER,
            "ACCESS_TOKEN": ACCESS_TOKEN,
            "API_USERNAME":"BOOMI_TOKEN.<BOOMI_USER>"
            }
    '''

    from dotenv import load_dotenv
    load_dotenv()

    # Access environment variables
    ACCOUNT_ID = os.getenv("ACCOUNT_ID")
    BOOMI_USER = os.getenv("BOOMI_USER")
    ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")

    credentials = {
        "ACCOUNT_ID": ACCOUNT_ID,
        "BOOMI_USER": BOOMI_USER,
        "ACCESS_TOKEN": ACCESS_TOKEN,
        "API_USERNAME":f"BOOMI_TOKEN.{BOOMI_USER}"
        }

    return credentials

## Build Headers Object - buildHeadersObject

In [6]:
def buildHeadersObject(api_credentials: dict, custom_args: Optional[dict] = None) -> dict:
    '''
    Builds a default dictionary of HTTP Headers with the following values:

        Content-Type: application/json
        Accept: application/json
        Connection: close
        user: <user authentication string> (Built from .env variables)

    If the custom_args dictionary is included, any key: value pairs in the dict will be added to the default headers.
    If any of the above keys are included in custom_args, the user provided value will overwrite the defualt value.

    Parameters
    ----------
    api_credentials : dict
        Dictonary of Environment Variables
    custom_args : dict, optional
        Dictionary of custom key:value pairs to include in the header dict, by default None

    Returns
    -------
    dict
        The dictionary to be used as json headers for an HTTP request 
    '''

    
    if custom_args is None:
        custom_args = {}


    default_headers = {
    "Content-Type": "application/json",
    "Accept":"application/json",
    "user":f"BOOMI_TOKEN.{api_credentials['BOOMI_USER']}:{api_credentials['ACCESS_TOKEN']}",
    "Connection": "close"
    }

    for custom_key, custom_value in custom_args.items():
        default_headers[custom_key] = custom_value


    return default_headers

## Make Standard POST Request - postRequestJSON

In [7]:
def postRequestJSON(resource_path: str, request_body: str, custom_headers: Optional[dict] = None) -> requests.models.Response:
    '''
    Makes a standard POST request with JSON body

    Parameters
    ----------
    resource_path : str
        resouce path to append to the base url
    request_body : str
        stringified JSON request body
    custom_headers : Optional[dict], optional
        custom header values to add to default headers, by default None

    Returns
    -------
    requests.models.Response
        HTTP Response
    '''

    api_credentials = getEnvironmentVariables()
    
    if resource_path.startswith("/"):
        resource_path = resource_path[1:]

    full_url_path = f"https://api.boomi.com/api/rest/v1/{ACCOUNT_ID}/{resource_path}"

    headers_dict = buildHeadersObject(api_credentials, custom_headers)

    post_response = requests.post(url=full_url_path, data=request_body, headers=headers_dict, auth=(api_credentials['API_USERNAME'], api_credentials['ACCESS_TOKEN']))


    return post_response

## Build Query Body - buildQueryBody

In [4]:
def buildQueryBody(query_params: dict, query_operator: str = "and") -> str:
    '''
    Builds a nested query body for use in a POST query

    Parameters
    ----------
    query_params : dict
        argument:property pairs to use in the query body
    query_operator : str, optional
        query relationship operator, either "and" or "or", by default "and"
 

    Returns
    -------
    str
        The query body as a string
    '''

    nested_expr = []

    for query_prop, query_arg in query_params.items():
        if not isinstance(query_arg, type([])):
            query_arg = [query_arg]

        query_expr = {"argument" : query_arg,
        "operator":"EQUALS",
        "property":query_prop}         
        
        nested_expr.append(query_expr)


    return json.dumps({
        "QueryFilter": {
            "expression": {
                "operator": query_operator,
                "nestedExpression": nested_expr
            }
        }
    })

In [ ]:
def build_query_body(*query_groups: dict, query_operator: str = "and") -> str:
    """
    Builds a nested query body for use in a POST query

    Parameters
    ----------
    query_groups : dict
        argument:property pairs to use in the query body
    query_operator : str, optional
        query relationship operator, either "and" or "or", by default "and"

    Returns
    -------
    str
        The query body as a string
    """

    nested_expr = []

    for query_group in query_groups:
        query_expr = {
            "argument": query_group.get("argument", []),
            "operator": query_group.get("operator", "EQUALS"),
            "property": query_group.get("property", "")
        }
        nested_expr.append(query_expr)

    
    if len(nested_expr) == 1:
        return json.dumps({
            "QueryFilter": {
                "expression": nested_expr[0]
            }
        })
    
    return json.dumps({
        "QueryFilter": {
            "expression": {
                "operator": query_operator,
                "nestedExpression": nested_expr
            }
        }
    })

In [26]:
r = build_query_body(
    {"argument": ["YL"], "property": "name"},
    query_operator="and")


print(r)

[{'argument': ['YL'], 'operator': 'EQUALS', 'property': 'name'}]
{"QueryFilter": {"expression": {"argument": ["YL"], "operator": "EQUALS", "property": "name"}}}


In [5]:
buildQueryBody({"name":"and", "id":"1234"})

'{"QueryFilter": {"expression": {"operator": "and", "nestedExpression": [{"argument": ["and"], "operator": "EQUALS", "property": "name"}, {"argument": ["1234"], "operator": "EQUALS", "property": "id"}]}}}'

## Folder Traversal 

In [ ]:
def traverseSubfolders(parentFolderName: str) -> pd.DataFrame:

    folderStructure = pd.DataFrame(columns=["folderName", "folderId", "parentFolderName", "parentFolderId", "parsedFlag"])
    

# Tests

In [9]:
folder_query_body = buildQueryBody({"name":["IAM Integrations"]})

In [16]:
folder_query = postRequestJSON(resource_path="Folder/query", request_body=folder_query_body)

In [17]:
type(folder_query)

requests.models.Response

In [10]:
folder_query = postRequestJSON(resource_path="Folder/query", request_body=folder_query_body)
folder_query.reason

'OK'

In [11]:
folder_query.json()

{'@type': 'QueryResult',
 'result': [{'@type': 'Folder',
   'id': 'Rjo2MTc3MTY2',
   'name': 'IAM Integrations',
   'fullPath': 'YL/IAM Integrations',
   'deleted': False,
   'parentId': 'RjoxMjA2MzIw',
   'parentName': 'YL'}],
 'numberOfResults': 1}

In [21]:
sub_folder_query_body = buildQueryBody({"parentId":["RjoxMjA2MzIw"]})
sub_folder_query = postRequestJSON(resource_path="Folder/query", request_body=sub_folder_query_body)
sub_folder_query.reason

'Bad Request'

In [14]:
sub_folder_query.json()

{'@type': 'QueryResult',
 'result': [{'@type': 'Folder',
   'id': 'Rjo2MTYzNDEz',
   'name': 'Job Code Manager',
   'fullPath': 'YL/IAM Integrations/Job Code Manager',
   'deleted': False,
   'parentId': 'Rjo2MTc3MTY2',
   'parentName': 'IAM Integrations'},
  {'@type': 'Folder',
   'id': 'Rjo2MTc3MjU5',
   'name': '#Common',
   'fullPath': 'YL/IAM Integrations/#Common',
   'deleted': False,
   'parentId': 'Rjo2MTc3MTY2',
   'parentName': 'IAM Integrations'},
  {'@type': 'Folder',
   'id': 'Rjo2MjM0MjAx',
   'name': 'User Account Manager',
   'fullPath': 'YL/IAM Integrations/User Account Manager',
   'deleted': False,
   'parentId': 'Rjo2MTc3MTY2',
   'parentName': 'IAM Integrations'},
  {'@type': 'Folder',
   'id': 'Rjo2NjQ0NzYy',
   'name': 'Candidate Change to YL API',
   'fullPath': 'YL/IAM Integrations/Candidate Change to YL API',
   'deleted': False,
   'parentId': 'Rjo2MTc3MTY2',
   'parentName': 'IAM Integrations'},
  {'@type': 'Folder',
   'id': 'Rjo2ODYwODg0',
   'name': 'Miss

In [149]:
component_query_body = buildQueryBody({"folderId":"Rjo2MTYzNDEz"})
component_query_body

'{"QueryFilter": {"expression": {"operator": "and", "nestedExpression": [{"argument": ["Rjo2MTYzNDEz"], "operator": "EQUALS", "property": "folderId"}]}}}'

In [150]:
component_query = postRequestJSON(resource_path="ComponentMetadata/query", request_body=component_query_body)
component_query.reason

'OK'

In [151]:
component_query.json()

{'@type': 'QueryResult',
 'queryToken': 'toDeU1LjiGV8hZH+/ad3ZPt+W8S+UHYgy2v7chANj9WafrVvpGPeN+ggR77su23FMiSZZymlDir1z1WrpfHBEpJvlzdXMNDRPNtT4uJXJRYVsjtTmelHNj0EFmepf3vDOU2e4aho4GXH6ukrn6FL4k0xfLNlI/8/1v2DPoZIyGLaP6o6n6YWoUFg/7JROVE95EBfouxhBl3TuzN1tRv8f6NLq1471ApyjCQSsjq684bX6UuQLhnfA5Hbf1fM527eeREb9tqceurFJBtwN8+sFfMQMukLGGa/rfJ0shTbbjxz6BZmTQ8FZttofxWCZJj490w3SDtIdTNIR3kvfvCvgYwq4KWORSEMR99DrdErgDsYrke00+xRb2GJClFKlC4iZ5C92xnflAGyi3c8ncDrg0MgVc7idK1U9f+NQREOQzdjLwi4oH7UeAH/y/gXJCkjBAXyXo7GmWw9+CVoHGI3hqT2l/cNsUghLXJtIMQYrBFvhJpynFTCEXwfQwa8Xe6H3Y/CkuWr+LED4VgzqPT2NbLBla8gpeoZSzsDg7Eoia+pxGc6wzPyHpgbVwL6sSzVbbBx65oLs9Nr8Q7+es/KGcCkhp5qb1eQ2xkYB68vEUZEY0w3eXpHoN0agXL4tE5ZlPLZDUo3xzQ66i8h3++U18b44k+SXM3haYui3N9ppjjwYB4yHha4rW2rWNslfOCKdae10sj3QqXMZjQqSyESumLqdrnYpA2OW79sUaUk0Gpq366tthd6ykAgfEiJFBblQlozg3dYP5FVJLLw/xo36zyGuSaL7oMsuBuMf+O//ms810FzDV82ceaTAf7+CyiPc7ebPKV7nsQ/wjAlEXmtftWKTGuEId99WoZeLszjF6Lg9AXzhMYZ7qjZs9XkTiq6AOTb49+QHE+pqJ6ooYbx3n+WmdJ2gcWcR3yrXnd5BbTQPeMbCtSRh7Jd0YxHRm8